In [1]:
# BUILD CUSTOM LAMBDA Layers, CUSTOM KERAS LAYERS AND MODELS

In [9]:
# -*- coding: UTF-8 -*-
from __future__ import division
import tensorflow as tf

# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *

from keras import backend as K
from keras.layers import Layer
from keras.layers.convolutional import Conv3D, Conv3DTranspose

GPU_IDS = '1'
#current_gpu = choose_gpu_by_id(GPU_IDS)
#print(current_gpu)

search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


In [53]:
abs(1.)%1==0

True

In [54]:
# build slice interactive to pad/crop in one step
import numpy as np
temp_shape = np.array((10,10,10))
empty = np.zeros(temp_shape)
img_shape = np.array((15,8,9))
img = np.ones(img_shape)
diff = img_shape - temp_shape
print(diff)
diff = diff/2
print(diff)
diff = tuple((int(x),int(x)) if abs(x)%1==0 else (int(x),int(x)+1) for x in diff)
print(diff)

crop = tuple(slice(abs(d[0]),abs(d[1])+t,1) if d[1] > 0 else slice(None,None,1) for d,t in zip(diff,temp_shape))

pad = tuple(slice(abs(d[0]),t-abs(d[1]),1) if d[1] < 0 else slice(None,None,1) for d,t in zip(diff,temp_shape))

print(crop)
print(pad)
empty[pad] = img[crop]

empty.shape




[ 5 -2 -1]
[ 2.5 -1.  -0.5]
((2, 3), (-1, -1), (0, 1))
(slice(2, 13, 1), slice(None, None, 1), slice(0, 11, 1))
(slice(None, None, 1), slice(1, 9, 1), slice(None, None, 1))


ValueError: could not broadcast input array from shape (11,8,9) into shape (10,8,10)

In [62]:
import numpy as np
temp_shape = np.array((10,10,10))
empty = np.zeros(temp_shape)
img_shape = np.array((15,8,9))
img = np.ones(img_shape)
diff = img_shape - temp_shape
print(diff)

[ 5 -2 -1]


In [12]:
import numpy as np
import logging
temp_shape =(15,8,9)
target_shape=(10,10,10)
img = np.ones(temp_shape)
padded = pad_and_crop(img, target_shape)
print(padded.shape)
orig = pad_and_crop(padded, temp_shape)
print(orig.shape)

(10, 10, 10)
(15, 8, 9)


In [13]:
%%timeit
pad_and_crop(img)

20.3 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [24]:
[np.random.rand(random.randint(0,9), random.randint(0,9)) for i in range(5)]

[array([[0.05807925, 0.87913938, 0.2689524 , 0.59411846, 0.4752153 ,
         0.04479035, 0.48679211, 0.05908073, 0.02259369],
        [0.54299318, 0.98048016, 0.34515032, 0.28432311, 0.88249342,
         0.65236019, 0.07173703, 0.99035071, 0.48426553],
        [0.50606116, 0.56998084, 0.89481091, 0.52157982, 0.95678293,
         0.09761827, 0.69549744, 0.19912031, 0.84466653],
        [0.51377544, 0.69307193, 0.9527031 , 0.13213195, 0.22347649,
         0.78744716, 0.4671425 , 0.83713698, 0.43973386],
        [0.53498896, 0.45662402, 0.83328119, 0.3056392 , 0.07841027,
         0.64448762, 0.11611232, 0.0278658 , 0.36514913],
        [0.53520601, 0.4150704 , 0.39870117, 0.21899294, 0.73608936,
         0.72657855, 0.55630066, 0.44814277, 0.73781528],
        [0.49018425, 0.37410546, 0.47152566, 0.75408359, 0.44361466,
         0.72132013, 0.12271277, 0.98121923, 0.03612006],
        [0.88130854, 0.81717421, 0.54579282, 0.42597479, 0.18406432,
         0.79891496, 0.1017725 , 0.4951457

In [31]:
import random
lower = 5
upper = 19
target_shape = (10,10,10)
images = [np.random.rand(random.randint(lower,upper),random.randint(lower,upper),random.randint(lower,upper)) for i in range(5)]

img_nda, mask_nda,sax3d, saxtoax3d, a = map(lambda x: pad_and_crop(x, target_shape=target_shape),images)

In [29]:
list(print(i.shape) for i in cropped)

(10, 10, 10)
(10, 10, 10)
(10, 10, 10)
(10, 10, 10)
(10, 10, 10)


[None, None, None, None, None]

In [11]:

def pad_and_crop(ndarray, target_shape=(10,10,10)):
    
    empty = np.zeros(target_shape)
    target_shape= np.array(target_shape)
    logging.debug('input shape', ndarray.shape)
    logging.debug('target shape', target_shape)
    
    diff = ndarray.shape - target_shape
    
    # divide into summands to work with odd numbers
    d = list((int(x//2),int(x//2)) if x%2==0 else (int(np.floor(x/2)), int(np.floor(x/2)+1)) for x in diff)
    # replace the second slice parameter if it is None, which slice until end of ndarray
    d = list((abs(x),abs(y)) if y !=0 else (abs(x),None) for x,y in d)
    # create a bool list, negative numbers --> pad, else --> crop
    pad_bool = diff<0
    crop_bool = diff>0
    
    # create one slice obj for cropping and one for padding
    pad = list(i if b else (None, None) for i,b in zip(d,pad_bool))
    crop = list(i if b else (None, None) for i,b in zip(d,crop_bool))
    
    # Create one tuple of slice calls per pad/crop
    # crop or pad from dif:-dif if second param not None, else replace by None to slice until the end
    # slice params: slice(start,end,steps)
    pad = tuple(slice(i[0], -i[1]) if i[1]!=None else slice(i[0],i[1]) for i in pad)
    crop = tuple(slice(i[0], -i[1]) if i[1]!=None else slice(i[0],i[1]) for i in crop)
    #print('crop',crop)
    
    # crop and pad in one step
    empty[pad] = ndarray[crop]
    return empty

In [77]:
pad[diff<0]

TypeError: only integer scalar arrays can be converted to a scalar index

In [68]:
crop =(slice(2,-3), slice(None,None), slice(None,None))
pad = (slice(None,None), slice(1,-1), slice(1,None))
temp = img[crop]
print(temp.shape)
empty[pad] = img[crop]
print(empty.shape)

(10, 8, 9)


ValueError: could not broadcast input array from shape (10,8,9) into shape (10,8,0)

In [15]:
# create some testdata

v_3ds = [np.random.random((1, 16, 224, 224, 3)) for i in range(5)]
v_4d = np.stack(v_3ds, axis=1)
v_4d.shape
v_3ds[0].shape

(1, 5, 16, 224, 224, 3)

(1, 16, 224, 224, 3)

In [44]:
# Create a fake batch of a 3D stack
v_3ds = [np.empty((16, 10, 10, 3)) for i in range(12)]
[elem.fill(i) for i, elem in enumerate(v_3ds)]
v_3ds = np.stack(v_3ds, axis=1)
#v_3ds = np.stack(v_3ds)
v_3ds.shape

# swap axes
v_3ds = tf.transpose(v_3ds, [1, 0, 2, 3, 4])
v_3ds.shape
# get indicies
indicies = list(range(v_3ds.shape[0]))
print(indicies)

# create a list of slices
v_3ds = [s for s in v_3ds]

# zip together
zipped = zip(v_3ds, indicies)

zipped_shuffeld = tf.random.shuffle(zipped)

v3d, ind =zip*(zipped_shuffeld)


# evaluate

# shuffle back
v_3ds_shuffeled = 
data = (slice, ind) for slice, ind in zip(v3d, ind)
sorted_by_second = sorted(data, key=lambda tup: tup[1])
v3d, _ = zip*(sorted_by_second)

# swap axis

# evaluate


[None, None, None, None, None, None, None, None, None, None, None, None]

(16, 12, 10, 10, 3)

TensorShape([12, 16, 10, 10, 3])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


TensorShape([12, 16, 10, 10, 3])

<tf.Tensor: shape=(12,), dtype=int32, numpy=array([ 1,  9,  4,  5,  7, 11,  3, 10,  6,  0,  2,  8], dtype=int32)>

In [45]:
# apply 3D convolutions in a single session

input = tf.constant(v_3ds[0], dtype=tf.float32)
print(input.shape)
bias_init = tf.constant_initializer(0)

output = Conv3D(32, kernel_size=(3,3,3), strides=(2, 2, 2), padding='same')(input)

with tf.Session() as s:

    s.run(tf.global_variables_initializer())
    o = s.run(output)
    print(o.shape)


TypeError: Expected tensor with type tf.float32 not tf.float64

In [5]:
# apply a 3D convolution on each timestep in a for loop - not sure if that works inside a graph

def conv(input):
    
    result = []
    for i in range(input.shape[1]):
        result.append(Conv3D(32, kernel_size=(3,3,3), strides=(2, 2, 2), padding='same')(input[:,i,:,:,:]))
    
    return tf.stack(result, axis=1)

input = tf.constant(v_4d, dtype=tf.float32)
print(input.shape)
output = conv(input)


with tf.Session() as s:

    s.run(tf.global_variables_initializer())
    o = s.run(output)
    print(o.shape)

(1, 5, 16, 224, 224, 3)
(1, 5, 8, 112, 112, 32)


In [6]:

class Conv4d(Layer):
    
    def __init__(self, filters=2, kernel_size=(3,3,3),strides=(1,1,1), **kwargs):
        self.strides = strides
        self.kernel_size = kernel_size
        self.filters = filters
        super(Conv4d, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.conv3d = Conv3D(self.filters, kernel_size=self.kernel_size, strides=self.strides, padding='same')
        self.conv3dshape = tuple((input_shape[0], *input_shape[2:]))
        self.conv3d.build(self.conv3dshape)
        self._trainable_weights = self.conv3d.trainable_weights
        super(Conv4d, self).build(input_shape)

    
    def call(self, x):
        print('conv4d called with x = {}'.format(x.shape))
        result = []
        for i in range(x.shape[1]):
            vol_3d = x[:,i,:,:,:,:]
            result.append(self.conv3d(vol_3d))
        tensor = tf.stack(result, axis=1)
        print('conv4d call will return: {}'.format(tensor.shape))
        return tensor

    def compute_output_shape(self, input_shape):
        shape_b = input_shape[0]
        shape_t = input_shape[1]
        shape_3d = input_shape[2:-1]
        shape_3d = (shape_3d[0]//self.strides[0], shape_3d[1]//self.strides[1], shape_3d[2]//self.strides[2])
        shape = (shape_b, shape_t, *shape_3d, self.filters)
        print('conv4d output shape: {}'.format(shape))
        return tuple(shape)
    
class Conv4DTranspose(Layer):
    
    def __init__(self, filters=2, kernel_size=(3,3,3),strides=(1,1,1), **kwargs):
        self.strides = strides
        self.kernel_size = kernel_size
        self.filters = filters
        super(Conv4DTranspose, self).__init__(**kwargs)

    def build(self, input_shape):
        self.conv3dtranspose = Conv3DTranspose(self.filters, kernel_size=self.kernel_size, strides=self.strides, padding='same')
        self.conv3dtransposeshape = tuple((input_shape[0], *input_shape[2:]))
        self.conv3dtranspose.build(self.conv3dtransposeshape)
        self._trainable_weights = self.conv3dtranspose.trainable_weights
        super(Conv4DTranspose, self).build(input_shape)
    
    
    def call(self, x):
        print('transpose4d called with x = {}'.format(x.shape))
        result = []
        for i in range(x.shape[1]):
            vol_3d = x[:,i,:,:,:,:]
            result.append(self.conv3dtranspose(vol_3d))
        tensor = tf.stack(result, axis=1)
        print('transpose4d call will return: {}'.format(tensor.shape))
        return tensor

    def compute_output_shape(self, input_shape):
        
        shape_b = input_shape[0]
        shape_t = input_shape[1]
        shape_3d = input_shape[2:-1]
        shape_3d = (shape_3d[0]*self.strides[0], shape_3d[1]*self.strides[1], shape_3d[2]*self.strides[2])

        shape = (shape_b, shape_t, *shape_3d, self.filters)
        print('transpose4d output shape: {}'.format(shape))
        return tuple(shape)

In [7]:
from keras.models import Model
from keras.models import Input
from keras.layers import Flatten, Dense, Reshape
from keras.layers.wrappers import TimeDistributed
# create data
x = np.random.random((100,5,10,11,12,3))
print(x.shape)
y = x *2
#x = np.random.random((1,5,10,11,12,32))
print(y.shape)

input = Input((5,10,11,12,3), name='input')
print(input.shape)
shape = K.int_shape(input)
output = Conv4d(filters=32, name='conv4d')(input)
output = TimeDistributed(Conv3D(32, kernel_size=3, strides=1, padding='same'), name='time_conv')(output)
output = Flatten()(output)
output = Dense(50, activation='relu', name='dense1')(output)
output = Dense(shape[1] * shape[2] * shape[3] * shape[4] * shape[5], activation='relu', name='dense2')(output)
output = Reshape((shape[1], shape[2], shape[3], shape[4], shape[5]), name='reshape')(output)
output = TimeDistributed(Conv3DTranspose(3, kernel_size=3, strides=1, padding='same', name='time_transpose'))(output)
output = Conv4DTranspose(filters=32, name='transpose4d_1')(output)
output = Conv4DTranspose(filters=3, name='transpose4d_2')(output)

model = Model(inputs=[input], outputs=[output])
model.compile(optimizer='adam', loss='mse')
model.summary()

result = model.fit(x=x, y=y, epochs=5)


(100, 5, 10, 11, 12, 3)
(100, 5, 10, 11, 12, 3)
(?, 5, 10, 11, 12, 3)
conv4d called with x = (?, 5, 10, 11, 12, 3)
conv4d call will return: (?, 5, 10, 11, 12, 32)
conv4d output shape: (None, 5, 10, 11, 12, 32)
transpose4d called with x = (?, 5, 10, 11, 12, 3)
transpose4d call will return: (?, 5, ?, ?, ?, 32)
transpose4d output shape: (None, 5, 10, 11, 12, 32)
transpose4d called with x = (?, 5, ?, ?, ?, 32)
transpose4d call will return: (?, 5, ?, ?, ?, 3)
transpose4d output shape: (None, 5, 10, 11, 12, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 5, 10, 11, 12, 3)  0         
_________________________________________________________________
conv4d (Conv4d)              (None, 5, 10, 11, 12, 32) 2624      
_________________________________________________________________
time_conv (TimeDistributed)  (None, 5, 10, 11, 12, 32) 27680     
____________________________

In [88]:
input

<tf.Tensor 'input_50:0' shape=(?, 5, 10, 11, 12, 3) dtype=float32>

In [69]:
# apply a 3D convolution on each timestep in a for loop - not sure if that works inside a graph
from keras.models import Model
from keras.models import Input
from keras.layers import Lambda
def conv(input):
    print(input.shape)
    result = []
    for i in range(input.shape[1]):
        result.append(Conv3D(3, kernel_size=(3,3,3), strides=(1, 1, 1), padding='same')(input[:,i,:,:,:,:]))
    return tf.stack(result, axis=1)


def get_output_shape(input_shape):
    
    return tuple(input_shape)

# create data
x = np.random.random((1,5,10,11,12,3))
print(x.shape)
y = x *2
print(y.shape)

input = Input((5,10,11,12,3), name='input')
print(input.shape)
output = Lambda(conv, output_shape=get_output_shape)(input)
model = Model(inputs=[input], outputs=[output])
model.compile(optimizer = 'adam', loss='mse')
model.summary()

result = model.fit(x=x, y=y, epochs=5)






(1, 5, 10, 11, 12, 3)
(1, 5, 10, 11, 12, 3)
(?, 5, 10, 11, 12, 3)
(?, 5, 10, 11, 12, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 5, 10, 11, 12, 3)  0         
_________________________________________________________________
lambda_29 (Lambda)           (None, 5, 10, 11, 12, 3)  0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 1.3490
Epoch 2/5
1/1 [==============================] - 0s 1ms/step - loss: 1.3490
Epoch 3/5
1/1 [==============================] - 0s 2ms/step - loss: 1.3490
Epoch 4/5
1/1 [==============================] - 0s 1ms/step - loss: 1.3490
Epoch 5/5
1/1 [==============================] - 0s 1ms/step - loss: 1.3490


In [33]:
import tensorflow as tf
from tf.keras import layers

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    def __init__(self,
               latent_dim=32,
               intermediate_dim=64,
               name='encoder',
               **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
  """Converts z, the encoded digit vector, back into a readable digit."""

  def __init__(self,
               original_dim,
               intermediate_dim=64,
               name='decoder',
               **kwargs):
    super(Decoder, self).__init__(name=name, **kwargs)
    self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
    self.dense_output = layers.Dense(original_dim, activation='sigmoid')

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(tf.keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(self,
               original_dim,
               intermediate_dim=64,
               latent_dim=32,
               name='autoencoder',
               **kwargs):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim,
                               intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = - 0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructed


(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)


original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
vae.compile(optimizer, loss=keras.losses.mean_squared_error)
vae.summary()
#vae.fit(x_train, x_train, epochs=3, batch_size=64)




ModuleNotFoundError: No module named 'tf'

In [23]:
tf.__version__

'1.12.0'

In [16]:
keras.__version__

'2.2.4'

In [25]:
# create a config
EXPERIMENT = '3D/gcn/vae'
now = datetime.datetime.now()
# image params
DIM = [16, 224, 224]
SPACING = [6, 1.0,1.0] # used by sitk, opposite order than numpy or tensorflow!
# Greyscale images
IMG_CHANNELS = 1
# RV = 1 = Y[...,0] 
# Myo = 2 = Y[...,1] 
# LV = 3 = Y[...,2]
MASK_VALUES = [0, 1, 2, 3]  
MASK_CLASSES = len(MASK_VALUES)
ARCHITECTURE = '3D'
AUGMENT = False
SHUFFLE = True
AUGMENT_GRID = False
SEED = 42
BATCHSIZE =  2 # 64, 16, 1
SCALER = 'MinMax'
EPOCHS = 50

SINGLE_OUTPUT = True

MSE = False 
WEIGHTS = False

# path params
DATASET = 'gcn'  # 'acdc' # or 'tetra'
TRAIN_PATH = 'data/raw/tetra/2D/train/'
VAL_PATH = 'data/raw/tetra/2D/val/'
TEST_PATH = 'data/raw/tetra/2D/test/'

MODEL_PATH = os.path.join(os.path.join('models', EXPERIMENT), str(now.strftime("%Y-%m-%d_%H_%M")))
TENSORBOARD_LOG_DIR = os.path.join(os.path.join('reports/tensorboard_logs', EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))
CONFIG_PATH = os.path.join(os.path.join('reports/configs/',EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))
HISTORY_PATH = os.path.join(os.path.join('reports/history/',EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))



config = dict(((key, value) for key, value in locals().items()
               if key.isupper() and key not in ['HTML', 'K']))
config

{'GPU_IDS': '1',
 'EXPERIMENT': '3D/gcn/vae',
 'DIM': [16, 224, 224],
 'SPACING': [6, 1.0, 1.0],
 'IMG_CHANNELS': 1,
 'MASK_VALUES': [0, 1, 2, 3],
 'MASK_CLASSES': 4,
 'ARCHITECTURE': '3D',
 'AUGMENT': False,
 'SHUFFLE': True,
 'AUGMENT_GRID': False,
 'SEED': 42,
 'BATCHSIZE': 2,
 'SCALER': 'MinMax',
 'EPOCHS': 50,
 'SINGLE_OUTPUT': True,
 'MSE': False,
 'WEIGHTS': False,
 'DATASET': 'gcn',
 'TRAIN_PATH': 'data/raw/tetra/2D/train/',
 'VAL_PATH': 'data/raw/tetra/2D/val/',
 'TEST_PATH': 'data/raw/tetra/2D/test/',
 'MODEL_PATH': 'models/3D/gcn/vae/2019-10-17_10_22',
 'TENSORBOARD_LOG_DIR': 'reports/tensorboard_logs/3D/gcn/vae/2019-10-17_10_22',
 'CONFIG_PATH': 'reports/configs/3D/gcn/vae/2019-10-17_10_22',
 'HISTORY_PATH': 'reports/history/3D/gcn/vae/2019-10-17_10_22'}

In [26]:
from src.models.vae import get_vae

In [28]:
encoder, decoder, vae = get_vae(config)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 16, 224, 224, 0                                            
__________________________________________________________________________________________________
conv3d_12 (Conv3D)              (None, 8, 112, 112,  896         encoder_input[0][0]              
__________________________________________________________________________________________________
conv3d_13 (Conv3D)              (None, 4, 56, 56, 64 55360       conv3d_12[0][0]                  
__________________________________________________________________________________________________
conv3d_14 (Conv3D)              (None, 2, 28, 28, 12 221312      conv3d_13[0][0]                  
__________________________________________________________________________________________________
flatten_3 